In [1]:
import os
import math
import numpy as np
import pandas as pd
from ast import literal_eval
import pymc3
import itertools
import arviz as az
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
pd.options.mode.chained_assignment = None  # default='warn'

from helper_functions import pre_process_input_data, pre_process_eye_data

## Writing all successfull runs of an individual participant into a single file

In [18]:
subject_ids = ["AE07EM1", "AG11ER1", "AM19ER1", "AN10HA1", "AR02AA", "AS01RH1", "EH08OI1", "EI13RE1", "EL10RM1", "EU29TT1", "FR04AN", "IT02IT1", "LA22KK1", "LJ28VA1", "LL13AE1", "MO07LN1", "OC19AS1", "OK01UE", "OL01AC1", "ON27IN1", "OR13UN1", "RE25EK", "RO25EA1", "TE26EI1", "UD06AD", "VE21IR1", "ZC12VE"]


In [22]:
root_dir = os.getcwd()

data_dir = "/experimental_data/"

target_string = "output"
target_string_eye_tracking = "eye_tracking"
done_string = "done"

successfull_runs = []

for subdir, dirs, files in os.walk(root_dir+data_dir):
    for file in files:
        if done_string in file:
            successfull_runs.append(file)

In [23]:
snippets = []

for file_name in successfull_runs:
    temp = file_name.split("_")
    code = temp[0]
    exp_trial = temp[-1]
    exp_trial = exp_trial[:2]  # cut off .csv
    
    snippets.append([code, exp_trial])

In [24]:
successfull_runs_df = pd.DataFrame(snippets, columns = ['code', 'trial'])
successfull_runs_df

,code,trial
0,OK01UE,10
1,OK01UE,25
2,OK01UE,16
3,OK01UE,27
4,OK01UE,07
...,...,...
821,UD06AD,16
822,UD06AD,13
823,UD06AD,35
824,UD06AD,45


In [32]:
fixations_complete_df = pd.DataFrame()
saccades_complete_df = pd.DataFrame()

for id_code in np.unique(successfull_runs_df.code):
    
    file_name = f'experimental_eye_data_{id_code}.csv'
    eye_data_successfull_runs = pd.DataFrame()
    
    successfull_runs = list(successfull_runs_df.loc[successfull_runs_df['code'] == id_code].trial)
    
    path = root_dir + data_dir + str(id_code) + "/eye_data"
    for subdir, dirs, files in os.walk(path):
        for data_file in files:
            if ".csv" in str(data_file):

                temp = str(data_file).split("_")
                # extract features of run from file_name coding
                level = temp[4][0]
                drift = temp[4][1]
                input_noise = temp[4][2]
                exp_trial = temp[-1]
                exp_trial = exp_trial[:2]
                # check for trial number of successfull trials
                #print(exp_trial, successfull_runs)
                if exp_trial in successfull_runs:

                    #f"experimental_data/{id_code}/eye_data/{data_file}"
                    temp_data = pre_process_eye_data(pd.read_csv((f"experimental_data/{id_code}/eye_data/{data_file}"), index_col=False))
                    temp_data["ID"] = id_code
                    temp_data["level"] = level
                    temp_data["drift"] = drift
                    temp_data["input_noise"] = input_noise

                    eye_data_successfull_runs = pd.concat([eye_data_successfull_runs, temp_data])
    
    # concat fixation data
    temp_fixations = eye_data_successfull_runs[eye_data_successfull_runs.fixationOnset == 1]
    fixations_complete_df = pd.concat([fixations_complete_df, temp_fixations])
    
    # concat saccade data
    temp_saccades = eye_data_successfull_runs[eye_data_successfull_runs.saccadeOnset == 1]
    saccades_complete_df = pd.concat([saccades_complete_df, temp_saccades])
    
    # writing subject data to file
    eye_data_successfull_runs.to_csv(file_name, sep=',', index=False)

# writing fixation and saccade data to files individually
fixations_complete_df = fixations_complete_df.reset_index()
saccades_complete_df = saccades_complete_df.reset_index()

fixations_complete_df.to_csv('experimental_eye_data_fixations.csv', sep=',')
saccades_complete_df.to_csv('experimental_eye_data_saccades.csv', sep=',')

# Metrics for situational anaylsis

We will focus on the usual suspects, that are: fixation_location_y, fixation_duration, and saccade_amplitude. But we will also assess the total number of fixations (n_fixations) and the total number of saccades (n_saccades) in a given time window.

Else we will investigate more the saccade directionality, meaning that we will assess saccadic landing sites. Specifically we will test if saccades land closer to obstacles, boarders (progressive saccades) or closer to the spaceship (regressive saccades).

# Analysing crash situations

First, we need to find for every participant (code) all runs in which a crash occured.

In [44]:
root_dir = os.getcwd()

data_dir = "/experimental_data/"

target_string = "output"
target_string_eye_tracking = "eye_tracking"
crash_string = "crashed"

crash_runs = []

for subdir, dirs, files in os.walk(root_dir+data_dir):
    for file in files:
        if crash_string in file:
                crash_runs.append(file)
            
snippets = []

for file_name in crash_runs:
    # extract features of run from file_name coding
    temp = file_name.split("_")
    
    code = temp[0]
    level = temp[2][0]
    drift = temp[2][1]
    input_noise = temp[2][2]
    exp_trial = temp[-1]
    exp_trial = exp_trial[:2]  # cut off .csv
    
    if level != '0':
        snippets.append([code, level, drift, input_noise, exp_trial])
    
crash_runs_df = pd.DataFrame(snippets, columns = ['code', 'level', 'drift', 'input_noise', 'trial'])
crash_runs_df

,code,level,drift,input_noise,trial
0,OK01UE,5,T,W,17
1,OK01UE,2,F,W,08
2,OK01UE,3,T,S,09
3,OK01UE,6,T,S,46
4,OK01UE,5,T,W,05
...,...,...,...,...,...
581,UD06AD,6,T,S,50
582,UD06AD,6,T,W,30
583,UD06AD,6,T,S,47
584,UD06AD,5,T,N,41


I have to find all the crash trials that have been successfully run through afterwards. For these cases, we can actually see the learning case-wise

## Putting together dataframe which holds crash runs and success runs

In [45]:
end_times = []
successful_runs_bool = []
success_runs = []


for index, row in crash_runs_df.iterrows():
    # storing important information in variables
    id_code = row['code']
    level = row['level']
    drift = row['drift']
    input_noise = row['input_noise']
    run = row['trial']
    
    root_dir = os.getcwd()
    data_dir = "/experimental_data/"
    eye_path = root_dir + data_dir + str(id_code) + "/eye_data"
    input_path = root_dir + data_dir + str(id_code) + "/data"
    
    data_file = f'{id_code}_eye_tracking_output_{level}{drift}{input_noise}_{run}.csv'
    
    # insert information in dataFrame
    eye_data = pre_process_eye_data(pd.read_csv((f"experimental_data/{id_code}/eye_data/{data_file}"), index_col=False))
    eye_data["ID"] = id_code
    eye_data["level"] = level
    eye_data["drift"] = drift
    eye_data["input_noise"] = input_noise

    # we need the points in time that are in the input_data file
    input_data_file_name = f"{id_code}_output_{level}{drift}{input_noise}_crashed_{run}.csv"
    input_data = pre_process_input_data(pd.read_csv((f"{input_path}/{input_data_file_name}"), index_col=False))
    end_time = input_data.time_played.iloc[-2]
    end_times.append(end_time)
    
    # check for successful run
    successful_run_filename = f'{id_code}_output_{level}{drift}{input_noise}_done_'
    
    for subdir, dirs, files in os.walk(input_path):
            for data_file in files:
                if str(successful_run_filename) in str(data_file):
                    successful_run_exist = True
                    
                    temp = data_file.split("_")
                    exp_trial = temp[-1]
                    exp_trial = exp_trial[:2]
                    print(exp_trial)
                    break
                else:
                    successful_run_exist = False
                    exp_trial = None
    successful_runs_bool.append(successful_run_exist)
    success_runs.append(exp_trial)
    
crash_runs_df['endTime'] = end_times
crash_runs_df['successfulRunBool'] = successful_runs_bool
crash_runs_df['successRun'] = success_runs


In [51]:
crash_runs_df

,code,level,drift,input_noise,trial,endTime,successfulRunBool,successRun
0,OK01UE,5,T,W,17,8.399715,False,None
1,OK01UE,2,F,W,08,30.822634,True,18
2,OK01UE,3,T,S,09,10.878068,True,14
3,OK01UE,6,T,S,46,37.392159,False,None
4,OK01UE,5,T,W,05,7.841097,False,None
...,...,...,...,...,...,...,...,...
581,UD06AD,6,T,S,50,14.523285,False,None
582,UD06AD,6,T,W,30,14.548611,True,34
583,UD06AD,6,T,S,47,11.295782,False,None
584,UD06AD,5,T,N,41,9.353312,False,None


### filtering only for crash runs with later success

In [52]:
crash_success_runs = crash_runs_df[crash_runs_df.successfulRunBool==True]
crash_success_runs

,code,level,drift,input_noise,trial,endTime,successfulRunBool,successRun
1,OK01UE,2,F,W,08,30.822634,True,18
2,OK01UE,3,T,S,09,10.878068,True,14
5,OK01UE,5,T,S,03,8.509098,True,15
9,OK01UE,6,T,W,42,30.569000,True,50
10,OK01UE,5,T,S,01,12.295899,True,15
...,...,...,...,...,...,...,...,...
567,UD06AD,5,F,S,43,21.823988,True,49
568,UD06AD,3,T,S,01,9.181145,True,23
570,UD06AD,6,T,W,17,14.891723,True,34
580,UD06AD,5,F,S,38,21.845131,True,49


In [56]:
# save progress
# crash_success_runs.to_csv('crash_success_runs.csv', index=False, sep=',')

Now that we have all the runs in which participants crashed but successfully ran through at a later point in the experiment, we can extract the metrics (metrics_crash and metrics_success) from the data files.

In [142]:
test_df = crash_success_runs.iloc[:2]

In [143]:
test_df

,code,level,drift,input_noise,trial,endTime,successfulRunBool,successRun
1,OK01UE,2,F,W,08,30.822634,True,18
2,OK01UE,3,T,S,09,10.878068,True,14


In [146]:
# 
time_window = 5

root_dir = os.getcwd()
data_dir = "/experimental_data/"
eye_path = root_dir + data_dir + str(id_code) + "/eye_data"
input_path = root_dir + data_dir + str(id_code) + "/data"

# initiate empty arrays for metrics
## crash
Nfixations_crash = []
fixDurs_crash = []
fixLocsY_crash = []
Nsaccades_crash = []
saccAmps_crash = []
distSpaceship_fix_rest_crash = []
distClosestObstacle_sacc_progress_crash = []
distClosestObstacle_sacc_regress_crash = []
distClosestObstacle_fix_explore_crash = []
## success
Nfixations_success = []
fixDurs_success = []
fixLocsY_success = []
Nsaccades_success = []
saccAmps_success = []
distSpaceship_fix_rest_success = []
distClosestObstacle_sacc_progress_success = []
distClosestObstacle_sacc_regress_success = []
distClosestObstacle_fix_explore_success = []


#for index, row in crash_success_runs.iterrows():
for index, row in crash_success_runs.iterrows():
    eye_file_name = f'{row.code}_eye_tracking_output_{row.level}{row.drift}{row.input_noise}_{row.trial}.csv'
    input_file_name = f'{row.code}_output_{row.level}{row.drift}{row.input_noise}_crashed_{row.trial}.csv'
    
    eye_data = pre_process_eye_data(pd.read_csv((f"experimental_data/{row.code}/eye_data/{eye_file_name}"), index_col=False))
    input_data = pre_process_input_data(pd.read_csv((f"experimental_data/{row.code}/data/{input_file_name}"), index_col=False))
    
    # subset data within range
    startTime = max(0, row.endTime - time_window)
    eye_data_ = eye_data[eye_data.time_tag.between(startTime, row.endTime)]
    input_data_ = input_data[input_data.time_played.between(startTime, row.endTime)]
    
    # extracting metrics from subsetted data
    Nfixations_crash.append(sum(eye_data_.fixationOnset))
    fixDurs_crash.append(np.array(eye_data_[eye_data_.fixationOnset == 1].fixation_duration))
    fixLocsY_crash.append(np.array(eye_data_[eye_data_.fixationOnset == 1].converging_eye_y_adjusted))
    Nsaccades_crash.append(sum(eye_data_.saccadeOnset))
    saccAmps_crash.append(np.array(eye_data_[eye_data_.saccadeOnset == 1].saccade_amplitude))
    distSpaceship_fix_rest_crash.append(np.array(get_dist_to_spaceship_fix_rest(eye_data=eye_data_, input_data=input_data_)))
    distClosestObstacle_sacc_progress_crash.append(np.array(get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='progress')))
    distClosestObstacle_sacc_regress_crash.append(np.array(get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='regress')))
    distClosestObstacle_fix_explore_crash.append(np.array(get_dist_to_obstacles_fix_explore(eye_data=eye_data_, input_data=input_data_)))
    
    # for successful runs ###################
    eye_file_name = f'{row.code}_eye_tracking_output_{row.level}{row.drift}{row.input_noise}_{row.successRun}.csv'
    input_file_name = f'{row.code}_output_{row.level}{row.drift}{row.input_noise}_done_{row.successRun}.csv'
    
    eye_data = pre_process_eye_data(pd.read_csv((f"experimental_data/{row.code}/eye_data/{eye_file_name}"), index_col=False))
    input_data = pre_process_input_data(pd.read_csv((f"experimental_data/{row.code}/data/{input_file_name}"), index_col=False))
    
    # subset data within range
    startTime = max(0, row.endTime - time_window)
    eye_data_ = eye_data[eye_data.time_tag.between(startTime, row.endTime)]
    input_data_ = input_data[input_data.time_played.between(startTime, row.endTime)]
    
    # extracting metrics from subsetted data
    Nfixations_success.append(sum(eye_data_.fixationOnset))
    fixDurs_success.append(np.array(eye_data_[eye_data_.fixationOnset == 1].fixation_duration))
    fixLocsY_success.append(np.array(eye_data_[eye_data_.fixationOnset == 1].converging_eye_y_adjusted))
    Nsaccades_success.append(sum(eye_data_.saccadeOnset))
    saccAmps_success.append(np.array(eye_data_[eye_data_.saccadeOnset == 1].saccade_amplitude))
    distSpaceship_fix_rest_success.append(np.array(get_dist_to_spaceship_fix_rest(eye_data=eye_data_, input_data=input_data_)))
    distClosestObstacle_sacc_progress_success.append(np.array(get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='progress')))
    distClosestObstacle_sacc_regress_success.append(np.array(get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='regress')))
    distClosestObstacle_fix_explore_success.append(np.array(get_dist_to_obstacles_fix_explore(eye_data=eye_data_, input_data=input_data_)))
    
    
    ##distClosestWall_sacc_progress = 
    ##distClosestWall_fix_explore = 
    
# insert metrics in dataFrame
crash_success_runs['Nfixations_crash'] = Nfixations_crash
crash_success_runs['fixDurs_crash'] = fixDurs_crash
crash_success_runs['fixLocsY_crash'] = fixLocsY_crash
crash_success_runs['Nsaccades_crash'] = Nsaccades_crash
crash_success_runs['saccAmps_crash'] = saccAmps_crash
crash_success_runs['distSpaceship_fix_rest_crash'] = distSpaceship_fix_rest_crash
crash_success_runs['distClosestObstacle_sacc_progress_crash'] = distClosestObstacle_sacc_progress_crash
crash_success_runs['distClosestObstacle_sacc_regress_crash'] = distClosestObstacle_sacc_regress_crash
crash_success_runs['distClosestObstacle_fix_explore_crash'] = distClosestObstacle_fix_explore_crash

crash_success_runs['Nfixations_success'] = Nfixations_success
crash_success_runs['fixDurs_success'] = fixDurs_success
crash_success_runs['fixLocsY_success'] = fixLocsY_success
crash_success_runs['Nsaccades_success'] = Nsaccades_success
crash_success_runs['saccAmps_success'] = saccAmps_success
crash_success_runs['distSpaceship_fix_rest_success'] = distSpaceship_fix_rest_success
crash_success_runs['distClosestObstacle_sacc_progress_success'] = distClosestObstacle_sacc_progress_success
crash_success_runs['distClosestObstacle_sacc_regress_success'] = distClosestObstacle_sacc_regress_success
crash_success_runs['distClosestObstacle_fix_explore_success'] = distClosestObstacle_fix_explore_success


In [151]:
#crash_success_runs.to_csv('crash_success_runs.csv', index=False, sep=',')
#crash_success_runs = pd.read_csv("crash_success_runs.csv", index_col=False)
crash_success_runs


,code,level,drift,input_noise,trial,endTime,successfulRunBool,successRun,Nfixations_crash,fixDurs_crash,...,distClosestObstacle_fix_explore_crash,Nfixations_success,fixDurs_success,fixLocsY_success,Nsaccades_success,saccAmps_success,distSpaceship_fix_rest_success,distClosestObstacle_sacc_progress_success,distClosestObstacle_sacc_regress_success,distClosestObstacle_fix_explore_success
1,OK01UE,2,F,W,08,30.822634,True,18,13,"[0.1624999999994543, 0.22549999999955617, 0.20...",...,[463400.7818985358],12,"[0.4025000000001455, 0.30099999999947613, 0.58...","[347.63775634765625, 596.971923828125, 370.779...",8,"[106.69255489313386, 125.4578634673037, 135.88...","[6118.213056010194, 106936.10283477698, 10303....",[5324.815530899912],"[250939.68181768805, 2558.7044190205634, 26105...","[79.35541790444404, 469247.0573369581]"
2,OK01UE,3,T,S,09,10.878068,True,14,10,"[0.3680000000003929, 0.3404999999993379, 0.191...",...,[],10,"[0.1860000000006039, 0.2335000000002765, 0.664...","[236.83721923828125, 581.8130493164062, 291.71...",8,"[203.97827211125838, 202.52507115051992, 50.40...","[1266.9413225846365, 98940.73587986547, 1412.3...","[4295.464391130954, 15029.024680890143]","[15119.49682531599, 25827.12938800454, 21330.9...",[]
5,OK01UE,5,T,S,03,8.509098,True,15,13,"[0.2084999999997308, 0.32700000000022555, 0.15...",...,"[141357.39749065135, 396408.64722275734, 780.1...",6,"[0.6310000000003129, 0.12000000000080036, 0.29...","[311.13348388671875, 608.3488159179688, 316.07...",2,"[252.21089198673624, 255.49142759404648]","[2504.272640801966, 225289.10015342012, 2389.0...",[167709.15525924042],[184728.23746127728],[]
9,OK01UE,6,T,W,42,30.569000,True,50,10,"[0.2360000000007858, 2.3759999999992942, 0.264...",...,"[6198.907223944552, 523.3290069438517]",13,"[0.1554999999998472, 0.22800000000006548, 0.06...","[369.8096923828125, 666.1378784179688, 603.161...",9,"[260.0583073822322, 30.267040517407015, 162.10...","[14255.17557920888, 164046.23432872165, 43520....","[1856.238147802651, 14131.7473419616, 42329.13...","[6646.178377185017, 3994.87994958926, 9146.626...","[17085.896602082998, 58141.87840582803, 8102.5..."
10,OK01UE,5,T,S,01,12.295899,True,15,8,"[0.4290000000000873, 0.7835000000004584, 0.478...",...,[],6,"[1.3519999999998618, 0.10949999999957072, 0.34...","[348.45599365234375, 416.7244873046875, 389.97...",3,"[356.7556372908118, 97.4282044374235, 7.558918...","[9710.039593853056, 21573.172498613596, 14425....",[],"[43719.01831271127, 20364.01121560484, 10165.5...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,UD06AD,5,F,S,43,21.823988,True,49,11,"[0.47299999999995634, 0.2444999999988795, 0.37...",...,[2143.857447432354],10,"[0.3469999999997526, 0.2039999999997235, 0.217...","[562.978515625, 504.00238037109375, 622.553009...",1,[282.55062204232075],"[86105.2282977933, 63902.4278158145, 128472.98...",[],[14382.957897622138],[3456.7091041477397]
568,UD06AD,3,T,S,01,9.181145,True,23,13,"[0.28650000000016007, 0.16350000000056752, 0.6...",...,"[630.6031190725043, 44872.75766785536, 14351.8...",9,"[0.4804999999996653, 0.2775000000001455, 0.198...","[453.4346923828125, 580.08935546875, 324.34381...",5,"[107.2878092761616, 130.03565983734353, 112.61...","[33721.03683652822, 96251.9206509823, 3492.576...",[113322.30823268928],"[30341.61701548472, 80085.67273948807, 22899.9...",[8896.878352830186]
570,UD06AD,6,T,W,17,14.891723,True,34,14,"[0.21550000000024738, 0.23300000000017462, 0.2...",...,"[34947.70690402761, 263.7422816641629, 660.326...",12,"[0.12899999999899592, 0.20499999999992724, 0.0...","[567.5956726074219, 402.372314453125, 452.3820...",6,"[56.19072987139764, 312.7528441714201, 183.413...","[90908.042618284, 19392.654258310795, 33271.05...",[],"[1268.2760043628514, 9761.058844325133, 11320....","[20732.05667242501, 5008.295361609198, 1996.35..."
580,UD06AD,5,F,S,38,21.845131,True,49,10,"[0.09399999999914144, 0.3845000000001164, 1.51...",...,[],10,"[0.3469999999997526, 0.2039999999997235

In [95]:
def get_dist_to_spaceship_fix_rest(eye_data, input_data):
    """
    eye_data and input_data must be already subsetted to the desired time period
    """
    # empty list to-be-returned
    dists = []
    
    timeTags = eye_data[eye_data.fixationOnset == 1].time_tag

    eyeX = eye_data[eye_data.fixationOnset == 1].converging_eye_x_adjusted
    eyeY = eye_data[eye_data.fixationOnset == 1].converging_eye_y_adjusted
    
    exploring = eye_data[eye_data.fixationOnset == 1].exploring_fixation

    # putting together temp_df
    temp = {'timeTag': timeTags, 'eyeX': eyeX, 'eyeY': eyeY, 'exploring_fixation': exploring}
    temp_df = pd.DataFrame(data=temp)
    
    # only resting fixations are of interest for dist to spaceship
    temp_df_rest = temp_df[temp_df["exploring_fixation"] == 0]
    
    for index, row in temp_df_rest.iterrows():
        # get row from input_data_ with closest match in time
        input_row = input_data.iloc[(input_data['time_played'] - row.timeTag).abs().argsort()[:1]]

        # get player position n pixel
        playerPosX = input_row.player_pos.values[0][0]
        playerPosY = input_row.player_pos.values[0][1]

        # compute distance to spaceship for each regressive saccade
        dist = np.power(playerPosX - row.eyeX, 2) + np.power(playerPosY - row.eyeY, 2)
        dists.append(dist)
    
    return dists

In [94]:
dists = get_dist_to_spaceship_fix_rest(eye_data=eye_data_, input_data=input_data_)
dists

       timeTag         eyeX        eyeY  exploring_fixation
7308    3.6540   921.243011  667.547852                   1
7824    3.9120   983.892914  219.326843                   0
8284    4.1420  1006.208862  436.501221                   0
8786    4.3930   989.023071  345.240295                   0
9196    4.5980  1373.656982  639.894501                   0
9626    4.8130   971.033173  345.057495                   0
10374   5.1870   967.284943  314.195801                   0
12186   6.0930   968.481873  472.007324                   0
13122   6.5610  1015.817810  327.468506                   0
14290   7.1450  1000.608032  540.748108                   0
14583   7.2915   962.671661  425.086060                   0
14824   7.4120   977.677399  330.014465                   0
16032   8.0160   961.081848  528.230835                   0
16461   8.2305   964.329956  369.575684                   0


[3461.3551103742793,
 30448.42179878056,
 6887.717575926334,
 312933.92456752155,
 5923.756542340852,
 2129.7585072973743,
 41016.68367082253,
 7124.070806149393,
 75476.8466049619,
 24126.883584083058,
 4162.355257418938,
 66733.31669776514,
 10022.024755254388]

In [138]:
def get_dist_to_obstacles_fix_explore(eye_data, input_data):
    """
    eye_data and input_data must be already subsetted to the desired time period.
    """
    # empty list to-be-returned
    dists = []
    
    timeTags = eye_data[eye_data.fixationOnset == 1].time_tag

    eyeX = eye_data[eye_data.fixationOnset == 1].converging_eye_x_adjusted
    eyeY = eye_data[eye_data.fixationOnset == 1].converging_eye_y_adjusted
    
    exploring = eye_data[eye_data.fixationOnset == 1].exploring_fixation

    # putting together temp_df
    temp = {'timeTag': timeTags, 'eyeX': eyeX, 'eyeY': eyeY, 'exploring_fixation': exploring}
    temp_df = pd.DataFrame(data=temp)

    # only exploring fixations are of interest for dist to obstacles
    temp_df_explore = temp_df[temp_df["exploring_fixation"] == 1]
    
    for index, row in temp_df_explore.iterrows():
        # get row from input_data_ with closest match in time
        input_row = input_data.iloc[(input_data['time_played'] - row.timeTag).abs().argsort()[:1]]
        
        # compute distances of all obstacles on screen to saccade landing site
        obsDists = []
        for obstacle in input_row.visible_obstacles.values[0]:
            dist = np.power(obstacle[0] - row.eyeX, 2) + np.power(obstacle[1] - row.eyeY, 2)
            obsDists.append(dist)
        # the shortest distance is the obstacle most likely in focus of visual attention
        if len(obsDists) > 0:
            dists.append(min(obsDists))
    
    return dists

In [100]:
dists = get_dist_to_obstacles_fix_explore(eye_data=eye_data_, input_data=input_data_)
dists

[9198.35815831367]

In [137]:
def get_dist_to_obstacles_sacc(eye_data, input_data, target_saccades='regress'):
    """
    eye_data and input_data must be already subsetted to the desired time period.
    target_saccades can either be = 'regress' (default) or = 'progress'
    """
    # empty list to-be-returned
    dists = []
    
    timeTags = eye_data[eye_data.saccadeOnset == 1].time_tag

    eyeX = eye_data[eye_data.saccadeOnset == 1].converging_eye_x_adjusted
    eyeY = eye_data[eye_data.saccadeOnset == 1].converging_eye_y_adjusted

    saccDirX = eye_data[eye_data.saccadeOnset == 1].saccade_direction_x
    saccDirY = eye_data[eye_data.saccadeOnset == 1].saccade_direction_y

    # putting together temp_df
    temp = {'timeTag': timeTags, 'eyeX': eyeX, 'eyeY': eyeY, 'saccDirX': saccDirX, 'saccDirY': saccDirY}
    temp_df = pd.DataFrame(data=temp)

    # flagging regressive saccades
    temp_df['regressiveSaccade'] = np.where(temp_df['saccDirY']< 0, True, False)
    
    temp_df['saccadeLandX'] = temp_df.eyeX + temp_df.saccDirX
    temp_df['saccadeLandY'] = temp_df.eyeY + temp_df.saccDirY
    
    # filtering for target saccades
    if target_saccades == 'regress':
        temp_df_targets = temp_df[temp_df.regressiveSaccade == True]
    elif target_saccades == 'progress':
        temp_df_targets = temp_df[temp_df.regressiveSaccade == False]
    
    for index, row in temp_df_targets.iterrows():
        # get row from input_data_ with closest match in time
        input_row = input_data.iloc[(input_data['time_played'] - row.timeTag).abs().argsort()[:1]]
        
        # compute distances of all obstacles on screen to saccade landing site
        obsDists = []
        for obstacle in input_row.visible_obstacles.values[0]:
            dist = np.power(obstacle[0] - row.saccadeLandX, 2) + np.power(obstacle[1] - row.saccadeLandY, 2)
            obsDists.append(dist)
        # the shortest distance is the obstacle most likely in focus of visual attention
        if len(obsDists) > 0:
            dists.append(min(obsDists))
            
    return dists

In [97]:
dists = get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='regress')
dists

[80686.84096083883,
 103390.85611463338,
 285270.81739132944,
 23343.852699115872,
 19824.86128574703,
 7991.533490636386]

In [98]:
dists = get_dist_to_obstacles_sacc(eye_data=eye_data_, input_data=input_data_, target_saccades='progress')
dists

[31459.780632690527, 211835.388324772]

In [77]:
id_code = crash_runs_df.code.iloc[0]
run = crash_runs_df.trial.iloc[0]

root_dir = os.getcwd()
data_dir = "/experimental_data/"
eye_path = root_dir + data_dir + str(id_code) + "/eye_data"
input_path = root_dir + data_dir + str(id_code) + "/data"

for subdir, dirs, files in os.walk(eye_path):
        for data_file in files:
            if str(run) in str(data_file):
                
                temp = str(data_file).split("_")
                # extract features of run from file_name coding
                level = temp[4][0]
                drift = temp[4][1]
                input_noise = temp[4][2]
                exp_trial = temp[-1]
                exp_trial = exp_trial[:2]

                #f"experimental_data/{id_code}/eye_data/{data_file}"
                eye_data = pre_process_eye_data(pd.read_csv((f"experimental_data/{id_code}/eye_data/{data_file}"), index_col=False))
                eye_data["ID"] = id_code
                eye_data["level"] = level
                eye_data["drift"] = drift
                eye_data["input_noise"] = input_noise

# we need the points in time that are in the input_data file
input_data_file_name = f"{id_code}_output_{level}{drift}{input_noise}_crashed_{run}.csv"
input_data = pre_process_input_data(pd.read_csv((f"experimental_data/{id_code}/data/{input_data_file_name}"), index_col=False))
end_time = input_data.time_played.iloc[-2]
                
end_time


8.399715423583984

In [60]:
eye_data

,TimeTag,LeftEyeX,LeftEyeY,LeftPupilDiameter,RightEyeX,RightEyeY,RightPupilDiameter,DigitalIn,LeftBlink,RightBlink,...,Saccade,saccadeOnset,N_saccade,saccade_direction_x,saccade_direction_y,saccade_amplitude,ID,level,drift,input_noise
0,16587.1830,-50.132446,-59.182861,43.492188,-101.882324,-203.447144,47.011719,16777215.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
1,16587.1835,-49.877686,-58.078186,43.476562,-101.181152,-201.540466,46.976562,16777215.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
2,16587.1840,-49.260803,-56.973145,43.445312,-101.158966,-199.677612,46.953125,16777215.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
3,16587.1845,-48.602417,-56.195557,43.468750,-100.553223,-196.629089,46.937500,16777215.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
4,16587.1850,-47.572632,-55.486267,43.457031,-100.624176,-193.677368,46.890625,16777215.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28303,16601.3345,NaN,NaN,9999.000000,NaN,NaN,9999.000000,16777215.0,1.0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
28304,16601.3350,NaN,NaN,9999.000000,NaN,NaN,9999.000000,16777215.0,1.0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
28305,16601.3355,NaN,NaN,9999.000000,NaN,NaN,9999.000000,16777215.0,1.0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S
28306,16601.3360,NaN,NaN,9999.000000,NaN,NaN,9999.000000,16777215.0,1.0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,MO07LN1,3,T,S


We need to define a window of time before the crash. Within this time window, we will run our analyses and compare it to aggregated data of successfull trials under the same set of conditions.

In visualize_sequence, I look at 5s. This window seems adequate. Maybe we will look at the first 5s before the crash and then the next 5s before that.

In [78]:
time_window = 5
start_time = max(0, end_time - time_window)

eye_data_ = eye_data[eye_data.time_tag.between(start_time, end_time)]
input_data_ = input_data[input_data.time_played.between(start_time, end_time)]


The simple metrics first...

In [6]:
N_fixations = sum(eye_data_.fixationOnset)
N_fixations

9

In [7]:
N_saccades = sum(eye_data_.saccadeOnset)
N_saccades

0

We might run into an issue here: what if there are no fixations or saccades?

In [8]:
fixDurs = eye_data_[eye_data_.fixationOnset == 1].fixation_duration
fixDurs

11299    0.5220
12454    0.0195
12522    0.0090
12602    0.1580
12973    0.9280
14903    0.9365
16849    0.2210
17340    1.4520
20288    0.4720
Name: fixation_duration, dtype: float64

In [9]:
fixLocs = eye_data_[eye_data_.fixationOnset == 1].converging_eye_y_adjusted
fixLocs

11299    351.225983
12454    467.962372
12522    489.614655
12602    537.421387
12973    431.321686
14903    442.547546
16849    514.796753
17340    414.009369
20288    351.185242
Name: converging_eye_y_adjusted, dtype: float64

In [10]:
saccAmps = eye_data_[eye_data_.saccadeOnset == 1].saccade_amplitude
saccAmps

Series([], Name: saccade_amplitude, dtype: float64)

Now the more complicated ones...

For each saccadeOnset, I will extract the point in time the saccadeOnset happened (time_tag) and the landing site of the saccade. I will search for the nearest row in the input_data in terms of time_played and extract the positions of all the obstacles and walls on screen, as well as the spaceship. In the following, I will then simply the squared distance of saccade landing site and the closest obstacle, or wall or spaceship (squared for penelizing higher distances and favoring smaller ones). The background is, that when the saccade is programmed, we hypothesize that saccades are programmed (first row of saccade) to land near visual markers for more accurate perception -> the decision-making is based more on visual perception, than on high-level cognitive abstraction.

For progressive saccades, we will check if obstacles were on screen and if that was the case, compute the distance to the closest obstacle. If however no obstacles were on screen, we will compute the distance to the closest wall.

In [15]:
# Extracting all the necessary data

# Fixations
timeTags = eye_data_[eye_data_.fixationOnset == 1].time_tag

eyeX = eye_data_[eye_data_.fixationOnset == 1].converging_eye_x_adjusted
eyeY = eye_data_[eye_data_.fixationOnset == 1].converging_eye_y_adjusted

exploreFix = eye_data_[eye_data_.fixationOnset == 1].exploring_fixation.astype(bool)

# Saccades
timeTags = eye_data_[eye_data_.saccadeOnset == 1].time_tag

eyeX = eye_data_[eye_data_.saccadeOnset == 1].converging_eye_x_adjusted
eyeY = eye_data_[eye_data_.saccadeOnset == 1].converging_eye_y_adjusted

saccDirX = eye_data_[eye_data_.saccadeOnset == 1].saccade_direction_x
saccDirY = eye_data_[eye_data_.saccadeOnset == 1].saccade_direction_y


In [ ]:
# Putting together the temporary fixation dataFrame
temp = {'timeTag': timeTags, 'eyeX': eyeX, 'eyeY': eyeY, 'saccDirX': saccDirX, 'saccDirY': saccDirY, 'saccAmp': saccAmps}
temp_df_crash = pd.DataFrame(data=temp)

# flagging regressive saccades
temp_df_crash['regressiveSaccade'] = np.where(temp_df_crash['saccDirY']< 0, True, False)

temp_df_crash

In [12]:
# Putting together the temporary saccade dataFrame
temp = {'timeTag': timeTags, 'eyeX': eyeX, 'eyeY': eyeY, 'saccDirX': saccDirX, 'saccDirY': saccDirY, 'saccAmp': saccAmps}
temp_df_crash = pd.DataFrame(data=temp)

# flagging regressive saccades
temp_df_crash['regressiveSaccade'] = np.where(temp_df_crash['saccDirY']< 0, True, False)

temp_df_crash


,timeTag,eyeX,eyeY,saccDirX,saccDirY,saccAmp,regressiveSaccade


Extracting the position of obstacles, walls and spaceship (allways the same) from input data at appropriate points in time and inserting them into temp_df_crash.


In [12]:
input_data_ = input_data[input_data.time_played.between(start_time, end_time)]
input_data_

,frame,trial,attempt,time_played,time_tag,level_size_y,player_pos,collision,current_input,drift_enabled,...,visible_obstacles,adjacent_wall_tiles_x_pos,visible_drift_tiles,SoC,adjusted_time_tag,start_input,N_input,input_change,drift_tile_onset,second_drift_tile_onset
307,307,3,1,5.239318,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[849, 204], [1047, 330], [921, 672]]",NaN,[],NaN,16592.423053,NaN,NaN,NaN,0,0
308,308,3,1,5.255999,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[849, 198], [1047, 324], [921, 666]]",NaN,[],NaN,16592.439734,NaN,NaN,NaN,0,0
309,309,3,1,5.272427,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[849, 192], [1047, 318], [921, 660]]",NaN,[],NaN,16592.456162,NaN,NaN,NaN,0,0
310,310,3,1,5.289043,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[849, 186], [1047, 312], [921, 654]]",NaN,[],NaN,16592.472778,NaN,NaN,NaN,0,0
311,311,3,1,5.306096,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[849, 180], [1047, 306], [921, 648]]",NaN,[],NaN,16592.489831,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,3,1,10.165401,16587.183735,9018,"[954, 270]",False,NaN,True,...,"[[739, 6], [1027, 24], [973, 294], [1153, 402]...",NaN,"[[436, 162], [436, 702]]",NaN,16597.349136,NaN,NaN,NaN,0,0
596,596,3,1,10.183625,16587.183735,9018,"[954, 270]",False,Left,True,...,"[[736, 0], [1024, 18], [970, 288], [1150, 396]...",NaN,"[[439, 156], [439, 696]]",NaN,16597.367360,1.0,6.0,0.0,0,0
597,597,3,1,10.203745,16587.183735,9018,"[954, 270]",False,Left,True,...,"[[1027, 12], [973, 282], [1153, 390], [829, 534]]",NaN,"[[439, 150], [439, 690]]",NaN,16597.387480,0.0,6.0,NaN,0,0
598,598,3,1,10.220882,16587.183735,9018,"[954, 270]",False,Left,True,...,"[[1027, 6], [973, 276], [1153, 384], [829, 528]]",NaN,"[[436, 144], [436, 684]]",NaN,16597.404617,0.0,6.0,NaN,0,0


In [13]:
temp_df_crash

,timeTag,eyeX,eyeY,saccDirX,saccDirY,saccAmp,regressiveSaccade


In [14]:
temp_df_crash['playerPosX'] = [np.nan]*len(temp_df_crash)
temp_df_crash['playerPosY'] = [np.nan]*len(temp_df_crash)
#temp_df_crash['wallPos'] = [np.nan]*len(temp_df_crash)
temp_df_crash['obstaclesPos'] = [np.nan]*len(temp_df_crash)
temp_df_crash

,timeTag,eyeX,eyeY,saccDirX,saccDirY,saccAmp,regressiveSaccade,playerPosX,playerPosY,obstaclesPos


In [15]:
temp_df_crash = temp_df_crash.reset_index()
temp_df_crash

,index,timeTag,eyeX,eyeY,saccDirX,saccDirY,saccAmp,regressiveSaccade,playerPosX,playerPosY,obstaclesPos


In [16]:
temp_df_crash['saccadeLandX'] = temp_df_crash.eyeX + temp_df_crash.saccDirX
temp_df_crash['saccadeLandY'] = temp_df_crash.eyeY + temp_df_crash.saccDirY

In [17]:
for index, row in temp_df_crash.iterrows():
    # get row from input_data_ with closest match in time
    input_row = input_data_.iloc[(input_data_['time_played'] - row.timeTag).abs().argsort()[:1]]

    # insert appropriate positions in temp_df
    temp_df_crash.at[index, 'playerPosX'] = input_row.player_pos.values[0][0]
    temp_df_crash.at[index, 'playerPosY'] = input_row.player_pos.values[0][1]
    #temp_df_crash.at[index, 'wallPos'] = input_row.last_walls_tile.values
    temp_df_crash.at[index, 'obstaclesPos'] = input_row.visible_obstacles.values[0]
    
    # compute distances of all obstacles on screen to saccade landing site
    obsDists = []
    for obstacle in input_row.visible_obstacles.values[0]:
        dist = np.power(obstacle[0] - temp_df_crash.at[index, 'saccadeLandX'], 2) + np.power(obstacle[1] - temp_df_crash.at[index, 'saccadeLandY'], 2)
        obsDists.append(dist)
    # choose and insert only shortest distance
    temp_df_crash.at[index, 'distanceClosestObstacle'] = min(obsDists)

In [18]:
temp_df_crash['distancePlayer'] = np.power(temp_df_crash.playerPosX - temp_df_crash.saccadeLandX, 2) + np.power(temp_df_crash.playerPosY - temp_df_crash.saccadeLandY, 2)


In [19]:
temp_df_crash

,index,timeTag,eyeX,eyeY,saccDirX,saccDirY,saccAmp,regressiveSaccade,playerPosX,playerPosY,obstaclesPos,saccadeLandX,saccadeLandY,distancePlayer
